In [15]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from itertools import chain
import time
import pandas as pd
import pickle as pk
from bs4 import BeautifulSoup
import requests
import os
import random

In [16]:
# Set EdgeDriver path
EDGEDRIVER_PATH = "./_edgedriver/msedgedriver.exe"
options = Options()
# Hide bot detection
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.headless = False  # False to visually check
service = Service(EDGEDRIVER_PATH)
driver = webdriver.Edge(service=service, options=options)

# Function to scroll down slowly


def slow_scroll():
    scroll_steps = random.randint(3, 7)  # Number of scroll steps
    for _ in range(scroll_steps):
        driver.execute_script("window.scrollBy(0, window.innerHeight / 2);")
        time.sleep(random.uniform(2, 5))  # Pause to mimic reading

# Function to get condo links


def get_all_links():
    links = []
    elements = driver.find_elements(By.CLASS_NAME, 'snippet')
    for elem in elements:
        try:
            href = elem.find_element(By.TAG_NAME, 'a').get_attribute("href")
            if href:
                links.append(href)
        except:
            continue
    return links


# Loop through pages with delays and scrolling
base_url = 'https://www.hipflat.co.th/en/condo-for-sale/bangkok?page={}'
all_condo_links = []

for page_num in range(1, 168):
    url = base_url.format(page_num)
    print(f"Scraping page {page_num}...")

    driver.get(url)

    # Scroll behavior to appear more human-like
    slow_scroll()

    # Random delay to prevent bot detection
    delay = random.uniform(5, 20)
    print(f"Waiting {delay:.2f} seconds...")
    time.sleep(delay)

    condo_links = get_all_links()
    all_condo_links.extend(condo_links)

    print(f"Found {len(condo_links)} links on page {page_num}")

# Save all links
with open("all_condo_links.txt", "w") as f:
    for link in all_condo_links:
        f.write(link + "\n")

print("Completed and saved all condo links.")
driver.quit()

Scraping page 1...
Waiting 9.71 seconds...
Found 30 links on page 1
Scraping page 2...


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: MicrosoftEdge=133.0.3065.82)
Stacktrace:
	GetHandleVerifier [0x00007FF6DFB3E735+13397]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6DFDCB1B4+2060276]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6DFD22338+1368440]
	(No symbol) [0x00007FF6DF8B9510]
	(No symbol) [0x00007FF6DF8D70CA]
	(No symbol) [0x00007FF6DF93A0E6]
	(No symbol) [0x00007FF6DF9513AD]
	(No symbol) [0x00007FF6DF933FC3]
	(No symbol) [0x00007FF6DF908B96]
	(No symbol) [0x00007FF6DF907E50]
	(No symbol) [0x00007FF6DF9089C3]
	(No symbol) [0x00007FF6DF990214]
	(No symbol) [0x00007FF6DFA206CF]
	(No symbol) [0x00007FF6DF993A53]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF6DFE845BD+279981]
	simdutf::get_active_implementation [0x00007FF6DFAC78C1+409457]
	simdutf::get_active_implementation [0x00007F

In [19]:
import requests
from bs4 import BeautifulSoup
import time
import random

# Base URL for pagination
base_url = "https://www.hipflat.co.th/en/condo-for-sale/bangkok?page={}"

# Fake browser headers (Important to avoid blocking)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    # Makes request look like it's from Google
    "Referer": "https://www.google.com/",
}

# Function to extract condo links from a page
def get_condo_links(page_num):
    url = base_url.format(page_num)
    print(f"Scraping page {page_num}...")

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if "verifying you are human" in response.text.lower():
            print("Blocked! Adding delay and retrying...")
            time.sleep(random.uniform(30, 60))  # Long delay before retry
            return get_condo_links(page_num)  # Retry the same page

        soup = BeautifulSoup(response.text, "html.parser")
        links = []

        # Find all condo listing links
        for a_tag in soup.select("div.snippet a[href]"):
            link = a_tag["href"]
            if link.startswith("https://www.hipflat.co.th/en/condo-for-sale/bangkok"):
                links.append(link)

        print(f"✅ Found {len(links)} links on page {page_num}")
        return links

    except requests.exceptions.RequestException as e:
        print(f"Error on page {page_num}: {e}")
        return []


# Loop through all pages (1-167) with random delays
all_condo_links = []
for page in range(1, 168):
    condo_links = get_condo_links(page)
    all_condo_links.extend(condo_links)

    # Random delay between requests to look human
    delay = random.uniform(5, 15)
    print(f"Waiting {delay:.2f} seconds before next request...")
    time.sleep(delay)

# Save links to a text file
with open("all_condo_links.txt", "w") as f:
    for link in all_condo_links:
        f.write(link + "\n")

print(f"🎉 Completed! Total condo listings: {len(all_condo_links)}")

Scraping page 1...
✅ Found 0 links on page 1
Waiting 5.96 seconds before next request...


KeyboardInterrupt: 